In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import image_dataset_from_directory
import cv2


2024-09-08 16:04:17.249465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-08 16:04:17.249568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-08 16:04:17.374933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
c=cv2.imread("/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image/Normal/10.jpg")
c.shape

(299, 299, 3)

In [4]:
df_lung="/kaggle/input/lung-disease/Lung X-Ray Image/Lung X-Ray Image"
training_data=image_dataset_from_directory(
df_lung,
seed=123,
image_size=(256,256),
batch_size=32,
subset="training",
validation_split=0.2)
validation_data=image_dataset_from_directory(
df_lung,
seed=123,
image_size=(256,256),
batch_size=32,
subset="validation",
validation_split=0.2)

Found 3475 files belonging to 3 classes.
Using 2780 files for training.
Found 3475 files belonging to 3 classes.
Using 695 files for validation.


# # Deep learning

In [5]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,Input
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [10]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [6]:
for img,label in training_data.take(2):
    print(img.shape,np.unique(label))

(32, 256, 256, 3) [0 1 2]
(32, 256, 256, 3) [0 1 2]


In [15]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
M=MaxPooling2D(2,2)(C)
B=BatchNormalization()(M)
D=Dropout(0.5)(B)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
M=MaxPooling2D(2,2)(C)
B=BatchNormalization()(M)
F=Flatten()(B)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(F)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=Dense(3,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])


In [16]:
model.fit(training_data,epochs=50,validation_data=validation_data,callbacks=[early_stopping])

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 220ms/step - accuracy: 0.6580 - loss: 7.5655 - val_accuracy: 0.4460 - val_loss: 11.6944
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 9s 103ms/step - accuracy: 0.8071 - loss: 4.9993 - val_accuracy: 0.4086 - val_loss: 6.6658
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - accuracy: 0.8564 - loss: 3.8181 - val_accuracy: 0.5309 - val_loss: 4.8474
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.8762 - loss: 3.2720 - val_accuracy: 0.6964 - val_loss: 3.3535
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - accuracy: 0.8854 - loss: 2.8771 - val_accuracy: 0.7122 - val_loss: 3.0429
Epoch 5: early stopping


In [17]:
loss,accuracy=model.evaluate(validation_data)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7212 - loss: 3.0290
